## Connect to template

In `server.py`, set -
```
add_routes(app, chain_ext, path="/rag_ollama_multi_query")
```

In [6]:
from langserve.client import RemoteRunnable

rag_app_ollama = RemoteRunnable("http://localhost:8000/rag-ollama-multi-query/")
rag_app_ollama.invoke ("What are the different types of agent memory?")


HTTPStatusError: Server error '500 Internal Server Error' for url 'http://localhost:8000/rag-ollama-multi-query/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500 for Internal Server Error

In [21]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from langchain_community.chat_models import ChatOllama

ollama_llm = "llama3:latest"
llm = ChatOllama(model=ollama_llm)

llm.invoke("Tell me about the history of AI")


AIMessage(content='What a fascinating topic!\n\nThe history of Artificial Intelligence (AI) spans several decades, with roots in ancient civilizations and rapid progress in the 20th century. Here\'s a brief overview:\n\n**Ancient Origins:**\n\n* The concept of artificial intelligence dates back to ancient Greece, where myths about creation of lifelike statues and automata existed.\n* In the 2nd century AD, the Greek philosopher Plato wrote about the idea of creating artificial beings that could think and act like humans.\n\n**19th-20th Century Developments:**\n\n* In the late 19th century, Charles Babbage proposed the concept of a "difference engine" that could perform calculations automatically.\n* In the early 20th century, Alan Turing\'s theoretical work on the Universal Turing Machine laid the foundation for modern computer science and AI.\n* The term "Artificial Intelligence" was coined in 1956 by John McCarthy, who also organized the first AI conference at Dartmouth College.\n\n*